# 목적

    - 데이터가 바이너리(엑셀) 형식으로 제공되면 파일형태로 다운로드후 수집 처리
    - 석유공사에서 제공하는 실시간 주유가격 사이트에서 지역별 가격 데이터 획득
    - 사이트는 시도 선택 -> 시군구 선택 -> 엑셀 저장 버튼 클릭후 다운로드된다
        - 엑셀을 다 모으면 파이썬에서 모든 파일을 읽어서 하나의 df로 구성 -> 저장
        - 모든 데이터 저장이 완료되면 엑셀 파일을 이동 혹은 삭제
    - ajax로 각 구간 통신이 진행된다 -> LEVEL 4 수준의 selenium 사용
    - 빈도 주기는 1일 1회 진행
        - 변경시간에 맞춰서 주기를 맞추면 된다

# step1 

    - 데이터를 수집하는 사이트까지 웹드라이버 가동후 페이지 접속하시오
    - https://www.opinet.co.kr/searRgSelect.do

In [1]:
import pandas as pd
import sys
from selenium import webdriver as wd
import time
import random

In [18]:
import selenium

selenium.__version__

'3.141.0'

In [2]:
driver = wd.Chrome(
    'C://Users//USER//Desktop//Py_Projects//selenium_level4//chromedriver.exe')

In [4]:
# 초기에 서브 페이지 접속을 거부하는 사이트도 있다 => 시간 텀을 두고 연속 접속 시도
target_site = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get( target_site )
time.sleep(2)
driver.get( target_site )

# step 2

    - 시도 select 태그를 찾아서 그밑에 시도를 표현하는 option태그의 value값들을 추출한다
    - #SIDO_NM0 > option

In [9]:
sidos = driver.find_elements_by_css_selector('#SIDO_NM0 > option')
# 프럼프트 1개 + 실제 데이터 17개
len(sidos)
for sido in sidos:
    # sido => <option ....>
    # 태그에서 속성값 추출할대 사용 : selenium에서는 get_attribute('value') <-> BS4에서는 get('value')
    name = sido.get_attribute('value').strip()
    if name:
        print( name )

In [13]:
sidos = [   sido.get_attribute('value').strip()
            for sido in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
            if sido.get_attribute('value').strip()
        ]
len( sidos )

17

In [14]:
sidos[8]

'경기도'

#  step3

    - 특정 시도 선택
    - send_keys() => 입력 계열의 모든 태그들에게 값을 설정할때 사용
        - input, select, textarea, checkbox, radio(?)

In [15]:
# 해당 시도를 선택 하는 이벤트 강제 발생
# 사람이 select 태그를 선택하면 select 태그의 value가 변경 
# -> 이벤트를 감지해서 -> ajax -> 시군구 전달 -> 화면 변경 : 5초대기
# ajax => cross domain이 처리를 기본적을 막는다(보안에 위배)
# 1. select 태그 특정
select_sido = driver.find_element_by_id('SIDO_NM0')
# 2. 값변경
select_sido.send_keys( sidos[8] )

In [17]:
# 전국 17개 시도를 돌면서 값변경(시도값) 진행
for sido in sidos:
    # 화면 조작시 그 대상이되면 메모리 주소가 변경된다 -> 알수없으므로 -> 매번 안전하게 획득
    select_sido = driver.find_element_by_id('SIDO_NM0')
    select_sido.send_keys( sido )
    time.sleep(5)
    print( sido )

서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원도
충청북도
충청남도
전라북도
전라남도
경상북도
경상남도
제주특별자치도


# step4
    - 시군구값들중 하나 선택
    - 잠시후
    - 엑셀저장 클릭

In [30]:
# 1차로 서울시의 자치구 이름(값)을 추출한다
sigungus = driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')

gus = list()
for sigungu in sigungus:
    gu = sigungu.get_attribute('value')
    if gu:
        gus.append( gu )

len(gus)

25

In [31]:
# 자치구 이름을 하나씩 추출
for gu in gus:    
    print( gu )
    # 해당 구 바로 선택 -> 값변경 -> 화면껌벅(form 전송됨) -> 해당구의 주유소 정보 세팅된다 :대기
    driver.find_element_by_id('SIGUNGU_NM0').send_keys( gu )
    time.sleep(10)
    # 클릭(엑셀 저장 버튼)
    driver.find_element_by_id('glopopd_excel').click()
    #break

강남구
강동구
강북구
강서구
관악구
광진구
구로구
금천구
노원구
도봉구
동대문구
동작구
마포구
서대문구
서초구
성동구
성북구
송파구
양천구
영등포구
용산구
은평구
종로구
중구
중랑구


# step5
    - 엑셀들을 다 모아서 읽어서 df로 구성

In [33]:
# 특정 폴더에 특정 형식의 파일들을 다 모아라(경로 + 이름(파일명))
import glob

files = glob.glob('C://Users//USER//Downloads//*.xls')
len(files), files[0]

(25, 'C://Users//USER//Downloads\\지역_위치별(주유소) (1).xls')

# xls -> DataFrame


In [38]:
import pandas as pd

# 상위 row 데이터 2개는 skip, 3번째 row 컬럼, 4번째부터 데이터

# df들을 담는 그릇
tmp = list()
# 파일 목록에서 파일 1개씩 추출
for file in files:
    # 파일 읽어서 -> df 생성 -> 그릇에 담는다
    tmp.append( pd.read_excel( file, header=2 ) )
# 총개수
len(tmp)

# df 25개를 생성 하여 1개의 df로 병합

25

In [39]:
# df를 하나로 합치기
full_df = pd.concat(tmp)
full_df.shape
# 서울시 주유소는 총 443개 존재한다, 10개의 속성(컬럼, 특징)을 표현한것

(443, 10)

In [40]:
full_df.head(1)

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),현대오일뱅크,02-487-2030,Y,-,1539,1742,-
